![alt text](images/HDAT9500Banner.PNG)
<br>

# Chapter 8: Sequential Data
# Exercise: Hidden Markov Models (HMM)


# 1. Introduction

In this exercise, we will build a Hidden Markov Model (HMM) to model sequential data (in particular, DNA sequences). We will replicate the HMM that appears in the Nature Paper: "What is a hidden Markov Model?" by Sean R Eddy that you can find in your repo or here:(https://www.nature.com/articles/nbt1004-1315.pdf)

Below is a graphical representation of a HMM:

![alt text](images/HMM_to_replicate.PNG)



## 1.1. Aims of the Exercise:
 1. To introduce HMMs.
 2. To explore sequential data.
 3. To create an HMM.
 4. To use an HMM to work in an **unsupervised** manner (**unsupervised learning**). HMM can be used in a supervised way too, but we are not going to study the supervisde version in this lesson.
 
It aligns with all the learning outcome of our course: 

1.	Distinguish a range of task specific machine learning techniques appropriate for Health Data Science.
2.	Design machine learning tasks for Health Data Science scenarios.
3.	Construct appropriate training and test sets for health research data.


## 1.2. Jupyter Notebook Intructions
1. Read the content of each cell.
2. Where necessary, follow the instructions that are written in each cell.
3. Run/Execute all the cells that contain Python code sequentially (one at a time), using the "Run" button.
4. For those cells in which you are asked to write some code, please write the Python code first and then execute/run the cell.
 
## 1.3. Tips
 1. The square brackets on the left hand side of each cell indicate whether the cell has been executed or not. Empty square brackets mean that the cell has not been excuted, whereas square brackets that contain a number means that the cell has been executed. Run all the cells in sequence, using the "Run" button.
 2. To edit this notebook, just double-click in each cell. In thid document, each cell can be a "Code" cell or "text-Markdown" cell. To choose between these two options, go to the combo-box above. 
 3. If you want to save your notebook, please make sure you press "the floppy disk" icon button above. 
 4. To clean the content of all cells and re-start Notebook, please go to Cell->All Output->Clear


# 2. HMM package: pomegranate
Following the example of the paper "What is a hidden Markov model?" (https://www.nature.com/articles/nbt1004-1315), let us create a HMM from scratch. 

We will use the Python library "pomegranate": 
    1. https://pomegranate.readthedocs.io/en/latest/
    2. https://media.readthedocs.org/pdf/pomegranate/latest/pomegranate.pdf (pages 47-64)

In [ ]:
import sys
print(sys.version)
#For this notebook to work, Python must be 3.6.4 or 3.6.5

import numpy as np
import pandas as pd
from IPython.display import display

from plotnine import *

In [ ]:
#!pip install pomegranate
from pomegranate import *

# 3. Some theory behind the Hiden Markov Model

## 3.1. Markov Property

From Wikipedia:

"A stochastic process has the Markov property if the conditional probability distribution of future states of the process (conditional on both past and present states) depends only upon the present state, not on the sequence of events that preceded it. A process with this property is called a Markov process.

The term Markov assumption is used to describe a model where the Markov property is assumed to hold, such as a hidden Markov model."

## 3.2. Hidden markov model

A hidden markov model contains two different types of states:
* *Hidden*, which we do not have access to. These are most often the values we would like to know. For our problem, the hidden states: exons, introns, or intergenic sequence.
* *Observable*, which we have access to. For our problem, the observed states are letters of the DNA sequence: CATGATTT

The intuition is that we cannot see the hidden states because this is what we are trying to predict. We use the observable states to make these predictions.<p>
    The probability of moving from one hidden state to another is called a *transition probability*. The probability of an observation given a specific hidden state is called *emission probabilities*. They are the probabilities that the observations are 'emitted' from the hidden state.<p>
        The connections between the hidden states and the observable states represent the probability of generating a particular observed state given that the Markov process is in a particular hidden state. 

The HMM is a generative probabilistic model, in which a sequence of observable **X** variables (DNA sequences, for example, ACGTA...) is generated by a sequence of internal hidden states **Z** (class: exons, introns, or intergenic sequence). The hidden states are not observed directly. The transitions between hidden states are assumed to be a Markov chain. A Markov chain is a model describing a sequence of possible events in which the probability of each event depends only on the state attained in the previous event. They can be specified by the start probability vector $\boldsymbol{\pi}$ and a transition probability matrix **A**. The emission probability of an observable can be any distribution with parameters $\boldsymbol{\theta}$ conditioned on the current hidden state. The HMM is completely determined by $\boldsymbol{\pi}$, **A**, and $\boldsymbol{\theta}$.<p>
    For our problem, we have no idea as to what the true values of $\boldsymbol{\pi}$, **A**, and $\boldsymbol{\theta}$ are. We ony have the observed data. So, our task will be to estimate the model parameters, and then use these estimated parameters to make predictions.

## 3.3. A simple example
Consider two friends, Bob and Dylan. They live very far away from each other, so Bob has no way of knowing directly what the weather is like where Dylan is. However he does know a few things:

* Bob can call Dylan and see how he is feeling today. He knows Dylan's mood is influenced by the weather. 
    * If it is sunny, he is happy with probability 0.8, and sad with probability 0.2.
* The weather today is dependent on the weather yesterday.
    * If it was sunny yestarday, it will be sunny tomorrow with probability 0.9
    * If it was rainy yesterday, it will be rainy today with probability 0.6.


This system describe a Hidden Markov Model, whereby:
* The hidden states are the weather condition (Bob cannot know these directly, just their transition probability)
* The observable states are Dylans mood (Bob can call Dylan, so he can directly measure his mood). Bob also knows the *emission probabilities*, which are the probabilities of a Dylan being in a particular mood given the weather condition.<p>

## 3.4.  Sequential (time-series) models
The key difference between sequential classification and regular classification (such as logistic regression, decision trees etc used for the hospital data set), is that we are able to include both the values of the feature variables **and** the order of the information.
<p>
    For example, consider a situation where we would like to predict if the weather is rainy or sunny based on a series of features. Now, we could use these features in isolation, which would produce a model that does not take into account the time stamps of our records. However, we can incorporate additional information by using the sequence of weather states of the previous days. This makes sense, as from experience we all know that the weather is often dependent on the previous day. The chance of the weather being sunny tomorrow is higher if it is sunny today, compared to if it is rainy today. Sequential models, such as HMMs, attempt to take this time dependence into account. 

# 4. HMM tasks (Very Important)
There are essentially three tasks associated with a HMM:
* **Task 1 - Learning:** Given the observed data (training data), estimated the model parameters. <p>  Learning involves generating a HMM from a sequence of observations. The hardest problem associated with HMMs is to take a sequence of observations from a known set, known to represent a set of hidden states, and fit the most probable model. We use the **Baum-Welch** algorithm for this purpose, which is an **Expectation-Maximization (EM)** algorithm. This algorithm identifies **local** optimal parameters for the HMM. This step is called "Problem 3" in the paper https://www.cs.sjsu.edu/~stamp/RUA/HMM.pdf <p>

* **Taks 2** - Find the probability of an observed sequence given our HMM. This is known as evaluation. This step uses the **Forward** algorithm. We use the forward algorithm to calculate the probability of an observation sequence given a particular HMM, and then choose the most likely HMM to be our model. This step is called "Problem 1" in the paper https://www.cs.sjsu.edu/~stamp/RUA/HMM.pdf. <p>
    
* **Task 3** - Find the sequence of hidden states that most probably generated an observed sequence. This is known as decoding. For this process, we will use the **Viterbi** algorithm. When we have decoded the sequence an have our predictions, we will compare it with the test set. This step is called "Problem 2" in hte paper https://www.cs.sjsu.edu/~stamp/RUA/HMM.pdf.<p>
    <p>
    
    **In this particular exercise:**<p>
    
* **Task 1 is given to us in this case**: the paper gives us the HMM paremeters. Probably this is because they have analysed many DNA sequences in the past an they know the state and emission probabilities (see section 5).

* **Task 2** is **not** carried out in this exercise. 
    
* **Taks 3**: After having built our model, we will find the sequence of hidden states that most probably generated an observed sequence.

# 5. Building a HMM

From the Nature paper (https://www.nature.com/articles/nbt1004-1315): <p><p>
   
### A toy HMM: 5′ splice site recognition <p>
    
"As a simple example, imagine the following caricature of a 5′ splice-site recognition problem. Assume we are given a DNA/RNA sequence that begins in an exon, contains one 5′ splice site and ends in an intron. **The problem is to identify where the switch from exon to intron occurred—where the 5′ splice site (5′SS) is.**<p>

For us to guess intelligently, the sequences of exons, splice sites and introns must have different statistical properties. Let's imagine some simple differences: say that exons have a uniform base composition on average (25% each base), introns are A/T rich (say, 40% each for A/T, 10% each for C/G), and the 5′SS consensus nucleotide is almost always a G (say, 95% G and 5% A).<p>

Starting from this information, we can draw an HMM (Fig. 1). The HMM invokes three states, one for each of the three labels we might assign to a nucleotide: E (exon), 5 (5′SS) and I (intron). Each state has its own emission probabilities (shown above the states), which model the base composition of exons, introns and the consensus G at the 5′SS. Each state also has transition probabilities (arrows), the probabilities of moving from this state to a new state. The transition probabilities describe the linear order in which we expect the states to occur: one or more Es, one 5, one or more Is.<p>
    
You can find what a splice site mutation is here: https://en.wikipedia.org/wiki/Splice_site_mutation

![alt text](images/HMM_to_replicate.PNG)

<div class="alert alert-block alert-success">**Start Activity 1**</div>

### <font color='blue'>Taking into account the specifications given for the HMM given in the picture above and the documentation of HMM/pomegranate: https://media.readthedocs.org/pdf/pomegranate/latest/pomegranate.pdf:</font> <p>

### <font color='blue'> Question 1a:  Initialize a model called "DNA Decoficiation"</font> <p>
 <font color='green'> Tip: page 47 of the pomegranate documenation: https://media.readthedocs.org/pdf/pomegranate/latest/pomegranate.pdf </font>

In [ ]:
# Type Python code here:
model =

### <font color='blue'> Question 1b: Define the emission probabilities as discrete distributions. Follow the distributions specified in the paper (see picture above)</font> <p>
 <font color='green'> Tip: page 48 and page 50 of the pomegranate documenation: https://media.readthedocs.org/pdf/pomegranate/latest/pomegranate.pdf </font>

In [ ]:
# Type Python code here:


### <font color='blue'> Question 1c:  Link each emission probability to a specific state/hidden variable. The names for state will be:</font>
   * for distribution 1, state "E", 
   * for distribution 2, state "5", 
   * for distribution 3, state "I".
  <p><p>
 <font color='green'> Tip: page 50 of the pomegranate documenation: https://media.readthedocs.org/pdf/pomegranate/latest/pomegranate.pdf </font>

In [ ]:
# Type Python code here:


### <font color='blue'> Question 1d:  Add the states to the model and initialize the transition probabilities </font> 
 <font color='green'> Tip: page 50 of the pomegranate documenation: https://media.readthedocs.org/pdf/pomegranate/latest/pomegranate.pdf </font><p>

In [ ]:
# Type Python code here:


### <font color='blue'> Question 1e:  Define a list that contains the DNA sequence (the one that appears in the picture above)</font> <p>
 <font color='green'> Tip: page 50 of the pomegranate documenation: https://media.readthedocs.org/pdf/pomegranate/latest/pomegranate.pdf </font>

In [ ]:
# Type Python code here:
DNA_test=

### <font color='blue'> Question 1f:  Use Viterbi to extract the most probable sequence of states/hidden variables:</font> <p>
 <font color='green'> Tip: page 50 of the pomegranate documenation: https://media.readthedocs.org/pdf/pomegranate/latest/pomegranate.pdf </font>

In [ ]:
# Type Python code here:


### <font color='blue'> Question 1g:  This is very important. Because the HMM is a generative model, it means that we have created a model that can **generate** sequences that followed the probabilities defined in the model. Generate 20 samples with our model. These sequences will be used in the Assessment.</font> <p>
 <font color='green'> Tip: page 46 of the pomegranate documenation: https://media.readthedocs.org/pdf/pomegranate/latest/pomegranate.pdf </font>

In [ ]:
# Type Python code here:


### <font color='blue'> Question 1h: Make sure you store the 20 sequences in this format (see below): </font> <p>


In [ ]:
# Type Python code here:
sequences = [ np.array(list("TCCCGGTA")),
             np.array(list("")),
             np.array(list("")),
             np.array(list("")),
             np.array(list("")),
             np.array(list("")),
             np.array(list("")),
             np.array(list("")),
             np.array(list("")),
             np.array(list("")),
             np.array(list("")),
             np.array(list("")),
             np.array(list("")),
             np.array(list("")),
             np.array(list("")), 
             np.array(list("")),
             np.array(list("")),
             np.array(list("")),
             np.array(list("")),           
             np.array(list("")) ]

<div class="alert alert-block alert-warning">**End Activity 1**</div>